In [1]:
import chess
import chess.pgn
import sys
from utils import *
from opening_book_gen import *
from huffman import *
from functools import reduce

In [11]:
def get_index(square, is_white=True):
    idx = 63 - square
    return idx if is_white else 8*(7-(idx//8)) + idx%8

def get_move_evals(board):
    moves = board.legal_moves
    values = []
    for i, move in enumerate(moves):
        piece = board.piece_at(move.from_square).symbol()
        is_white = piece.isupper()
        current_val = pst[piece.upper()][get_index(move.from_square, is_white)]
        new_val = pst[piece.upper()][get_index(move.to_square, is_white)]
        prom_bon = move.promotion << 26 if move.promotion else 0
        cap_bon = board.piece_at(move.to_square).piece_type << 25 if board.piece_at(move.to_square) else 0
        atk_by_bon = (7-board.piece_at(move.from_square).piece_type if board.is_attacked_by(not is_white, move.to_square) else 6) << 22
        values.append(atk_by_bon + cap_bon + prom_bon + new_val - current_val)
    return sorted(zip(moves, values), key=lambda x: x[1], reverse=True)

def get_move_index(move, eval_rankings, book=None):
    if not isinstance(book, dict): book = {}
    rankings = list(dict(generate_rankings(book)).keys()) + [m for m, _ in eval_rankings]
    return list(dict.fromkeys(rankings)).index(move)

def load_games(count=sys.maxsize):
    fh = open("lichess_db_standard_rated_2013-02.pgn")
    games = [chess.pgn.read_game(fh) for _ in range(count)]
    return games

In [14]:
def encode(game, book=None):
    encoding = ""
    board = game.board()
    for n, move in enumerate(game.mainline_moves()):
        idx = get_move_index(move, get_move_evals(board), book)
        encoding += codes[idx]
        board.push(move)
        book = book.get(move, None) if isinstance(book, dict) else None
    return (encoding, n)

In [15]:
def benchmark(book, upto=200):
    tmvs, tbits = 0, 0
    for game in games[:upto]:
        c, mvs = encode(game, book)
        tmvs += mvs
        tbits += len(c)
    print(f"Avg bits/mv: {tbits/tmvs}")

In [16]:
games = load_games(2500)

In [17]:
benchmark(None) # 4.407 bits to beat (4.3 using opening book - 0.1 bits/mv)

Avg bits/mv: 4.407531892139638


In [18]:
benchmark(generate_fixed_depth_opening_book(games[200:], depth=4))
benchmark(generate_fixed_depth_opening_book(games[200:], depth=6))
benchmark(generate_fixed_depth_opening_book(games[200:], depth=10))

Avg bits/mv: 4.3692613245741345
Avg bits/mv: 4.351233671988389
Avg bits/mv: 4.340386525093575


In [19]:
benchmark(generate_variable_depth_opening_book(games[200:], 10))
benchmark(generate_variable_depth_opening_book(games[200:], 3))
benchmark(generate_variable_depth_opening_book(games[200:], 1))

Avg bits/mv: 4.362157207241617
Avg bits/mv: 4.34978229317852
Avg bits/mv: 4.342601787487587


In [ ]:
# Graph idea: opening book size (depth/MB) -> bits/mv
# Include fixed depth & dynamic depth
# Show game between average lichess people, smallest size in bits.

In [ ]:
# naive opening book & non-custom Huffman codes -> 0.1 bits/move improvement (current)
# naive opening book & custom Huffman codes -> 0.15 bits/move improvement
# non-naive opening book & custom Huffman: 0.3 bits/move??

# find move likely to be played based on criteria
# (opening-book, other features) -> P(played)
# move played 2 moves ago ranked higher if prev move same piece (i.e. repeats)
# ALT: all moves with piece moved last 2 moves ranked higher

# multiple huffman codes, based on p of geometric distribution
# generate on startup and reuse.
# quick: just uses mean of distribution
# can we hop from distribution mean straight to the code?

In [ ]:
TODO: 
    - repeat move piece heuristic
    - 